# 优化 LLM 部署的有效技术

大型语言模型（LLM）如GPT3/4、[Falcon](https://huggingface.co/tiiuae/falcon-40b)和[Llama](https://huggingface.co/meta-llama/Llama-2-70b-hf)在处理人类任务方面能力日益增强，成为现代知识型产业中的重要工具。然而，在实际任务中部署这些模型仍面临挑战：

- 为了接近人类的文本理解和生成能力，LLM需要包含数十亿个参数，这增加了推理所需的内存需求。
- 在许多实际任务中，LLM需要处理大量的背景信息，这对模型处理长输入序列的能力提出了要求。

本文将介绍优化LLM部署的有效技术：

1. **降低精度**：研究表明，使用8位或4位的低精度运算可以在不大幅降低模型性能的情况下获得计算优势。
2. **Flash注意力**：Flash注意力是一种更高效的注意力算法，能够有效利用GPU内存，提高效率。
3. **架构创新**：提出了一些专门针对长时间输入序列优化的模型架构，如Alibi、旋转位置嵌入（Rotary embeddings）、多查询注意力（MQA）和分组查询注意力（GQA）。

### 1. 降低精度

我们可以将LLM视为一系列权重矩阵和向量，文本输入为一系列向量。每个参数通常存储为`float32`、`bfloat16`或`float16`格式。例如，对于拥有X亿个参数的模型，在`float32`精度下加载权重需要大约4X GB的显存。

为了减少显存占用，现在大多数模型都是用`bfloat16`或`float16`训练的。因此，加载权重的显存需求变为2X GB。

对于较短的文本输入（少于1024个标记），推理所需的显存主要由加载权重占用。以下是几个模型所需的显存量：

- **GPT3**：2 * 175 GB = **350 GB**
- [**Bloom**](https://huggingface.co/bigscience/bloom)：2 * 176 GB = **352 GB**
- [**Llama-2-70b**](https://huggingface.co/meta-llama/Llama-2-70b-hf)：2 * 70 GB = **140 GB**
- [**Falcon-40b**](https://huggingface.co/tiiuae/falcon-40b)：2 * 40 GB = **80 GB**
- [**MPT-30b**](https://huggingface.co/mosaicml/mpt-30b)：2 * 30 GB = **60 GB**
- [**bigcode/starcoder**](https://huggingface.co/bigcode/starcoder)：2 * 15.5 = **31 GB**

目前市场上的最大GPU（如A100或H100）提供80GB显存，而大多数模型需要更多显存，因此需要使用张量并行或管道并行技术。

使用`device_map="auto"`可以自动将不同层分配到可用的GPU上。此外，可以使用[文本生成推理库](https://github.com/huggingface/text-generation-inference/tree/main/server/text_generation_server/models/custom_modeling)来编写支持张量并行的模型。

### 2. Flash注意力

Flash注意力通过在计算注意力时动态量化权重，减少了显存需求并提高了计算效率。对于长输入序列，Flash注意力显著降低了显存占用。

### 3. 架构创新

为了更高效地处理长时间输入序列，研究人员提出了一些改进模型架构的方法，如：

- **相对位置嵌入**（如RoPE和ALiBi），可以更好地处理长文本输入。
- **多查询注意力（MQA）**，减少了缓存占用。
- **分组查询注意力（GQA）**，在保持性能的同时进一步减少了内存占用。

#### 3.1 改进LLM的位置嵌入

绝对位置嵌入（如正弦位置嵌入和学习位置嵌入）对长文本输入效果较差。相对位置嵌入（如RoPE和ALiBi）通过在注意力层中直接编码句子顺序，使模型更好地理解相对位置关系。例如，RoPE通过旋转向量角度来编码相对位置，而ALiBi通过添加负整数来调整注意力分数。

#### 3.2 缓存键值对

自回归文本生成可以通过缓存先前的键值对来减少计算量。使用缓存后，每次只计算当前查询向量与缓存键值对的乘积，显著提高了推理速度并减少了显存占用。

#### 3.2.1 多轮对话

在多轮对话中，缓存键值对可以避免重复计算，使模型能够快速响应后续问题。例如，在多轮对话中，缓存第一轮对话的键值对，第二轮对话只需处理新输入部分，从而使推理速度更快。

#### 3.2.2 多查询注意力（MQA）

MQA通过共享所有注意力头的键值投影权重，显著减少了缓存占用。例如，对于16000个输入序列，使用MQA可以将缓存占用从15 GB减少到不到400 MB。

#### 3.2.3 分组查询注意力（GQA）

GQA通过使用少量键值投影权重（如2、4或8个），在保持性能的同时减少了内存占用。GQA还可以通过少量计算将现有模型转换为GQA架构。